Import Libraries/Data

In [1]:
import os
import sys
import glob
import numpy as np
import torch
from torchvision import transforms, datasets
from PIL import Image

#Import image loaders
os.chdir("../../src/")
from load_data import preview_images, load_images

In [2]:
#Import and preview image data
train_path= "../Data/train/"
horse_train= load_images(train_path + "horses/")
human_train= load_images(train_path + "humans/")

val_path= "../Data/validation/"
horse_test= load_images(val_path + "horses/")
human_test= load_images(val_path + "humans/")

#Sanity Check
print("Training Horse Data: %s \nTest Horse Data: %s\n" % (horse_train.shape, horse_test.shape))
print("Training Human Data: %s \nTest Human Data: %s" % (human_train.shape, human_test.shape))

Training Horse Data: (500, 300, 300, 4) 
Test Horse Data: (128, 300, 300, 4)

Training Human Data: (527, 300, 300, 4) 
Test Human Data: (128, 300, 300, 4)


Preprocess **Training** Data

In [3]:
#Assign labels: 1= human, 0 = horse
y_train_human= np.full((len(human_train),1), 1)
y_train_horse= np.full((len(horse_train),1), 0)

#Concatenate Training data
X= np.concatenate((horse_train, human_train), axis=0)
y= np.concatenate((y_train_horse, y_train_human), axis= 0)

#Zero-center data
training_mean= X.mean()
training_std= X.std()
X= (X - training_mean)/training_std

#Shuffle, reshape and convert to tensors
from sklearn.utils import shuffle
X_train, y_train= shuffle(X, y)

X_train= X_train.reshape((-1, 4, 300, 300))

X_train= torch.from_numpy(X_train)
y_train= torch.from_numpy(y_train)

#Sanity Check
print(X_train.shape)
print(y_train.shape)
#Wrap tensors into Dataset 
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)    

torch.Size([1027, 4, 300, 300])
torch.Size([1027, 1])


In [4]:
#Preprocess Test Data
y_test_human= np.full((len(human_test),1), 1)
y_test_horse= np.full((len(horse_test),1), 0)

#Concatenate validation set
X= np.concatenate((horse_test, human_test), axis=0)
y= np.concatenate((y_test_horse, y_test_human), axis= 0)

#Zero-Center data
X= (X- training_mean)/training_std

#Shuffle, reshape, and convert test data to tensors
X_test, y_test= shuffle(X, y)

X_test= X_test.reshape((-1, 4, 300, 300))

X_test= torch.from_numpy(X_test)
y_test= torch.from_numpy(y_test)

#Sanity Check
print("Test data shape: %s \nTest label shape: %s" % (X_test.shape, y_test.shape))

#Wrap tensors into Dataset 
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

Test data shape: torch.Size([256, 4, 300, 300]) 
Test label shape: torch.Size([256, 1])


# Modeling

Base ANN 

In [5]:
#Use base model from early computer vision
from models import BaseNeuralNetwork, train_loop, test_loop

device= "cpu"
base_model= BaseNeuralNetwork().to(device)
print(base_model)

BaseNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=360000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


Base ANN training

In [6]:
from torch import optim
from torch import nn
from torch.utils.data import DataLoader


#Set up loss function, optimizer and batch
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(base_model.parameters(),
                      lr= 10 ** -3,
                       momentum= .9)

batch_size= 64

#Create DataLoader iterable
train_dataloader= DataLoader(train_dataset, batch_size= batch_size, shuffle= False, drop_last= True)
test_dataloader= DataLoader(test_dataset, batch_size= len(test_dataset), shuffle= False)

#Sanity Check
for x, y in train_dataloader:
    print(x.shape)
    print(y.shape)
    break          
#Sanity Check
for x, y in test_dataloader:
    print(x.shape)
    print(y.shape)
    break 

torch.Size([64, 4, 300, 300])
torch.Size([64, 1])
torch.Size([256, 4, 300, 300])
torch.Size([256, 1])


In [ ]:
epochs = 3
for e in range(epochs):
    print(f"Epoch {e + 1}:\n----------------------------")
    train_loop(train_dataloader, base_model, criterion, optimizer)
    test_loop(test_dataloader, base_model, criterion)

Epoch 1:
----------------------------
loss: 0.697754  [    0/ 1027]
loss: 0.675377  [   64/ 1027]
loss: 0.623814  [  128/ 1027]
loss: 0.626251  [  192/ 1027]
loss: 0.601297  [  256/ 1027]
loss: 0.715795  [  320/ 1027]
loss: 0.516409  [  384/ 1027]
loss: 0.501635  [  448/ 1027]
loss: 0.439040  [  512/ 1027]
loss: 0.446919  [  576/ 1027]
loss: 0.432253  [  640/ 1027]
loss: 0.372604  [  704/ 1027]
loss: 0.354605  [  768/ 1027]
loss: 0.376940  [  832/ 1027]
loss: 0.320026  [  896/ 1027]
loss: 0.434664  [  960/ 1027]
Test Error: 
 Accuracy: 73.0%, Avg loss: 1.006387 

Epoch 2:
----------------------------
loss: 0.366628  [    0/ 1027]
loss: 0.289138  [   64/ 1027]
loss: 0.206712  [  128/ 1027]
loss: 0.329080  [  192/ 1027]
loss: 0.272741  [  256/ 1027]
loss: 0.402812  [  320/ 1027]
loss: 0.342310  [  384/ 1027]
loss: 0.222391  [  448/ 1027]
loss: 0.254685  [  512/ 1027]
loss: 0.254607  [  576/ 1027]
loss: 0.211176  [  640/ 1027]
loss: 0.248827  [  704/ 1027]
loss: 0.180324  [  768/ 1027]
lo

In [ ]:
len(test_dataloader.dataset)

In [ ]:
model= base_model
size= len(test_dataloader.dataset)
num_batches= len(test_dataloader)
test_loss, correct = 0, 0

with torch.no_grad():
        for (X, y) in test_dataloader:
            preds= model(X.float())

            test_loss += criterion(preds, y.flatten()).item()
            correct += (preds.argmax(1) == y).type(torch.float).sum().item()
            print(correct)
            print(y)